# RAG 基礎 (Retrieval-Augmented Generation)

**對應課程**: 李宏毅 2025 Fall GenAI-ML HW2, 2025 Spring ML HW1

RAG 是結合檢索與生成的技術，讓 LLM 能夠存取外部知識庫，解決知識截斷和幻覺問題。

## 學習目標
1. 理解 RAG 的核心架構與動機
2. 實作文件載入與切分
3. 使用向量嵌入（Embeddings）表示文本
4. 建立向量資料庫（FAISS）
5. 實作完整的 RAG Pipeline

## Part 1: 為什麼需要 RAG？

### 1.1 LLM 的限制

```
┌─────────────────────────────────────────────────────────────┐
│                     LLM 的三大限制                           │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  1. 知識截斷 (Knowledge Cutoff)                              │
│     • LLM 只知道訓練資料截止日期前的資訊                      │
│     • 無法回答最新事件或更新的知識                           │
│                                                             │
│  2. 幻覺 (Hallucination)                                    │
│     • 模型可能生成看似合理但實際錯誤的內容                    │
│     • 特別是專業領域或細節資訊                               │
│                                                             │
│  3. 無法存取私有資料                                         │
│     • 企業內部文件、個人筆記等                               │
│     • 需要特定領域知識的應用                                 │
│                                                             │
└─────────────────────────────────────────────────────────────┘

                    RAG 的解決方案
                         ↓
    ┌─────────────────────────────────────────────┐
    │  檢索相關文件 → 注入 Context → LLM 生成回答  │
    └─────────────────────────────────────────────┘
```

### 1.2 RAG 架構概覽

```
┌─────────────────────────────────────────────────────────────────────┐
│                         RAG Pipeline                                │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  ┌──────────────────── 離線索引階段 ────────────────────┐          │
│  │                                                       │          │
│  │   文件庫        切分           嵌入          向量 DB   │          │
│  │  ┌─────┐      ┌─────┐      ┌─────┐      ┌─────────┐  │          │
│  │  │Doc 1│  →   │Chunk│  →   │ Vec │  →   │ Vector  │  │          │
│  │  │Doc 2│      │Chunk│      │ Vec │      │  Store  │  │          │
│  │  │ ... │      │ ... │      │ ... │      │ (FAISS) │  │          │
│  │  └─────┘      └─────┘      └─────┘      └─────────┘  │          │
│  │                                              ↓       │          │
│  └──────────────────────────────────────────────────────┘          │
│                                                 │                   │
│  ┌──────────────────── 線上查詢階段 ────────────┼───────┐          │
│  │                                              │       │          │
│  │   使用者問題       嵌入          相似度搜尋   │       │          │
│  │  ┌─────────┐    ┌─────┐      ┌──────────┐   │       │          │
│  │  │Question │ →  │ Vec │  →   │ Top-K    │←──┘       │          │
│  │  └─────────┘    └─────┘      │ Retrieval│           │          │
│  │                              └────┬─────┘           │          │
│  │                                   ↓                 │          │
│  │                           ┌──────────────┐          │          │
│  │                           │ Retrieved    │          │          │
│  │                           │ Documents    │          │          │
│  │                           └──────┬───────┘          │          │
│  │                                  ↓                  │          │
│  │                    ┌────────────────────────┐       │          │
│  │                    │   Prompt = Question    │       │          │
│  │                    │         + Context      │       │          │
│  │                    └───────────┬────────────┘       │          │
│  │                                ↓                    │          │
│  │                          ┌──────────┐               │          │
│  │                          │   LLM    │               │          │
│  │                          └────┬─────┘               │          │
│  │                               ↓                     │          │
│  │                          ┌──────────┐               │          │
│  │                          │  Answer  │               │          │
│  │                          └──────────┘               │          │
│  └─────────────────────────────────────────────────────┘          │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# 環境設置
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import re
import json

# 檢查設備
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")

# 安裝提示
print("\n建議安裝的套件:")
print("  pip install sentence-transformers faiss-cpu")
print("  # 或 GPU 版本: pip install faiss-gpu")

## Part 2: 文件載入與切分

### 2.1 為什麼需要切分文件？

- LLM 有 context window 限制
- 長文件難以精確檢索
- 切分成小塊可以提高相關性匹配

In [ ]:
@dataclass
class Document:
    """文件資料結構"""
    content: str
    metadata: Dict = None
    
    def __post_init__(self):
        if self.metadata is None:
            self.metadata = {}

class TextSplitter:
    """文本切分器"""
    
    def __init__(self, chunk_size: int = 500, chunk_overlap: int = 50):
        """
        Args:
            chunk_size: 每個 chunk 的最大字元數
            chunk_overlap: 相鄰 chunk 的重疊字元數
        """
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
    
    def split_text(self, text: str) -> List[str]:
        """基礎切分：按字元數"""
        chunks = []
        start = 0
        
        while start < len(text):
            end = start + self.chunk_size
            chunk = text[start:end]
            chunks.append(chunk.strip())
            start = end - self.chunk_overlap
        
        return [c for c in chunks if c]  # 移除空 chunk
    
    def split_by_sentences(self, text: str) -> List[str]:
        """按句子切分，確保不會切斷句子"""
        # 簡單的句子分割（可以用更複雜的 NLP 工具）
        sentences = re.split(r'(?<=[.!?。！？])\s+', text)
        
        chunks = []
        current_chunk = []
        current_length = 0
        
        for sentence in sentences:
            sentence_length = len(sentence)
            
            if current_length + sentence_length > self.chunk_size and current_chunk:
                # 儲存當前 chunk
                chunks.append(' '.join(current_chunk))
                
                # 處理 overlap
                overlap_text = ' '.join(current_chunk)
                overlap_start = max(0, len(overlap_text) - self.chunk_overlap)
                overlap_sentences = overlap_text[overlap_start:].strip()
                
                current_chunk = [overlap_sentences] if overlap_sentences else []
                current_length = len(overlap_sentences)
            
            current_chunk.append(sentence)
            current_length += sentence_length + 1
        
        if current_chunk:
            chunks.append(' '.join(current_chunk))
        
        return chunks

# 測試切分
sample_text = """
Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience. 
It focuses on developing algorithms that can access data and use it to learn for themselves. 
The process begins with observations or data, such as examples, direct experience, or instruction. 
Deep learning is a type of machine learning that uses neural networks with many layers. 
These deep neural networks are capable of learning complex patterns in large amounts of data. 
Applications include image recognition, natural language processing, and autonomous vehicles.
Transformers have revolutionized natural language processing since their introduction in 2017.
They use self-attention mechanisms to process sequential data more efficiently than RNNs.
"""

splitter = TextSplitter(chunk_size=200, chunk_overlap=30)

print("=== 基礎切分 ===")
chunks = splitter.split_text(sample_text)
for i, chunk in enumerate(chunks):
    print(f"\nChunk {i+1} ({len(chunk)} chars):")
    print(f"  {chunk[:100]}..." if len(chunk) > 100 else f"  {chunk}")

print("\n=== 按句子切分 ===")
chunks_sentences = splitter.split_by_sentences(sample_text)
for i, chunk in enumerate(chunks_sentences):
    print(f"\nChunk {i+1} ({len(chunk)} chars):")
    print(f"  {chunk[:100]}..." if len(chunk) > 100 else f"  {chunk}")

### 2.2 切分策略比較

```
┌──────────────────────────────────────────────────────────────┐
│                     切分策略比較                              │
├────────────────┬─────────────────────────────────────────────┤
│ 策略            │ 優缺點                                       │
├────────────────┼─────────────────────────────────────────────┤
│ 固定長度        │ + 實作簡單                                   │
│                │ - 可能切斷語義                                │
├────────────────┼─────────────────────────────────────────────┤
│ 按句子          │ + 保持句子完整性                              │
│                │ - chunk 大小不均勻                            │
├────────────────┼─────────────────────────────────────────────┤
│ 按段落          │ + 保持段落語義完整                            │
│                │ - 段落可能太長                                │
├────────────────┼─────────────────────────────────────────────┤
│ 語義切分        │ + 最佳語義邊界                                │
│ (Semantic)     │ - 計算成本高                                  │
├────────────────┼─────────────────────────────────────────────┤
│ 遞迴切分        │ + 適應性強                                    │
│ (Recursive)    │ - 實作複雜                                    │
└────────────────┴─────────────────────────────────────────────┘
```

In [ ]:
class RecursiveTextSplitter:
    """遞迴文本切分器 - 類似 LangChain 的實作"""
    
    def __init__(self, chunk_size: int = 500, chunk_overlap: int = 50,
                 separators: List[str] = None):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.separators = separators or ["\n\n", "\n", ". ", " ", ""]
    
    def split_text(self, text: str) -> List[str]:
        """遞迴切分文本"""
        return self._split_text(text, self.separators)
    
    def _split_text(self, text: str, separators: List[str]) -> List[str]:
        """遞迴切分實作"""
        final_chunks = []
        
        # 找到第一個可用的分隔符
        separator = separators[-1]
        for sep in separators:
            if sep == "":
                separator = sep
                break
            if sep in text:
                separator = sep
                break
        
        # 使用分隔符切分
        if separator:
            splits = text.split(separator)
        else:
            splits = list(text)
        
        # 合併小片段
        current_chunk = []
        current_length = 0
        
        for split in splits:
            split_length = len(split)
            
            if current_length + split_length > self.chunk_size:
                if current_chunk:
                    chunk_text = separator.join(current_chunk)
                    
                    # 如果 chunk 太大，遞迴切分
                    if len(chunk_text) > self.chunk_size:
                        if separators[:-1]:
                            final_chunks.extend(self._split_text(chunk_text, separators[1:]))
                        else:
                            final_chunks.append(chunk_text[:self.chunk_size])
                    else:
                        final_chunks.append(chunk_text)
                    
                    current_chunk = []
                    current_length = 0
            
            current_chunk.append(split)
            current_length += split_length + len(separator)
        
        if current_chunk:
            final_chunks.append(separator.join(current_chunk))
        
        return [c.strip() for c in final_chunks if c.strip()]

# 測試遞迴切分
recursive_splitter = RecursiveTextSplitter(chunk_size=200, chunk_overlap=30)
chunks_recursive = recursive_splitter.split_text(sample_text)

print("=== 遞迴切分 ===")
for i, chunk in enumerate(chunks_recursive):
    print(f"\nChunk {i+1} ({len(chunk)} chars): {chunk[:80]}...")

## Part 3: 向量嵌入（Embeddings）

### 3.1 什麼是向量嵌入？

將文本映射到高維向量空間，使得語義相似的文本在向量空間中距離較近。

```
文本                              向量表示
"機器學習"    ────────→    [0.23, -0.41, 0.87, ...]
"深度學習"    ────────→    [0.25, -0.38, 0.82, ...]  ← 相似！
"今天天氣"    ────────→    [-0.52, 0.71, 0.12, ...] ← 不相似
```

In [ ]:
# 使用 sentence-transformers 進行嵌入
try:
    from sentence_transformers import SentenceTransformer
    
    # 載入模型（多語言支援）
    print("載入 embedding 模型...")
    embed_model = SentenceTransformer('all-MiniLM-L6-v2')  # 輕量級模型
    # 或使用更強的多語言模型: 'paraphrase-multilingual-MiniLM-L12-v2'
    
    print(f"模型載入完成")
    print(f"向量維度: {embed_model.get_sentence_embedding_dimension()}")
    
    # 測試嵌入
    test_sentences = [
        "Machine learning is a branch of artificial intelligence.",
        "Deep learning uses neural networks with many layers.",
        "The weather is sunny today.",
        "AI and ML are transforming industries.",
        "I like to eat pizza for dinner."
    ]
    
    embeddings = embed_model.encode(test_sentences)
    print(f"\n嵌入結果形狀: {embeddings.shape}")
    
except ImportError:
    print("請安裝 sentence-transformers: pip install sentence-transformers")
    embeddings = None

In [ ]:
# 計算並視覺化相似度矩陣
def compute_similarity_matrix(embeddings: np.ndarray) -> np.ndarray:
    """計算餘弦相似度矩陣"""
    # 正規化
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    normalized = embeddings / norms
    # 計算相似度
    similarity = np.dot(normalized, normalized.T)
    return similarity

if embeddings is not None:
    similarity_matrix = compute_similarity_matrix(embeddings)
    
    # 視覺化
    plt.figure(figsize=(10, 8))
    plt.imshow(similarity_matrix, cmap='RdYlBu_r', vmin=-1, vmax=1)
    plt.colorbar(label='Cosine Similarity')
    
    # 標註
    short_labels = ['ML/AI', 'Deep Learning', 'Weather', 'AI/ML transform', 'Pizza']
    plt.xticks(range(len(short_labels)), short_labels, rotation=45, ha='right')
    plt.yticks(range(len(short_labels)), short_labels)
    
    # 在格子中標註數值
    for i in range(len(similarity_matrix)):
        for j in range(len(similarity_matrix)):
            plt.text(j, i, f'{similarity_matrix[i,j]:.2f}', 
                    ha='center', va='center', fontsize=10)
    
    plt.title('文本向量相似度矩陣')
    plt.tight_layout()
    plt.show()
    
    print("\n觀察:")
    print("- ML/AI 與 Deep Learning 高度相似 (0.70+)")
    print("- Weather 和 Pizza 與技術主題不相似")
    print("- AI/ML transform 與前兩個也相似")

### 3.2 自訂 Embedding 包裝器

為了統一介面，我們建立一個 Embedding 類別：

In [ ]:
class EmbeddingModel:
    """Embedding 模型包裝器"""
    
    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        try:
            from sentence_transformers import SentenceTransformer
            self.model = SentenceTransformer(model_name)
            self.dimension = self.model.get_sentence_embedding_dimension()
        except ImportError:
            print("警告: 使用隨機嵌入作為替代")
            self.model = None
            self.dimension = 384  # 預設維度
    
    def embed_documents(self, texts: List[str]) -> np.ndarray:
        """嵌入多個文件"""
        if self.model:
            return self.model.encode(texts, show_progress_bar=False)
        else:
            # 隨機嵌入（僅作示範）
            np.random.seed(42)
            return np.random.randn(len(texts), self.dimension).astype(np.float32)
    
    def embed_query(self, text: str) -> np.ndarray:
        """嵌入單一查詢"""
        return self.embed_documents([text])[0]

# 建立 embedding 模型
embedding_model = EmbeddingModel()
print(f"Embedding 維度: {embedding_model.dimension}")

## Part 4: 向量資料庫（FAISS）

### 4.1 FAISS 介紹

FAISS (Facebook AI Similarity Search) 是一個高效的相似度搜尋庫。

```
┌─────────────────────────────────────────────────────────────┐
│                     FAISS 索引類型                           │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  IndexFlatL2     - 暴力搜尋（最準確，最慢）                   │
│  IndexFlatIP     - 內積搜尋                                  │
│  IndexIVFFlat    - 倒排索引（更快，需要訓練）                 │
│  IndexIVFPQ      - 量化壓縮（記憶體效率）                     │
│  IndexHNSW       - 圖搜尋（快速，高記憶體）                   │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
# FAISS 向量儲存
try:
    import faiss
    FAISS_AVAILABLE = True
except ImportError:
    FAISS_AVAILABLE = False
    print("FAISS 未安裝，使用簡化實作")

class VectorStore:
    """向量儲存與檢索"""
    
    def __init__(self, dimension: int):
        self.dimension = dimension
        self.documents: List[Document] = []
        
        if FAISS_AVAILABLE:
            # 使用 FAISS
            self.index = faiss.IndexFlatIP(dimension)  # 內積（需要正規化向量）
        else:
            # 簡化實作
            self.vectors = []
    
    def add_documents(self, documents: List[Document], embeddings: np.ndarray):
        """加入文件及其向量"""
        self.documents.extend(documents)
        
        # 正規化向量（用於餘弦相似度）
        norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
        normalized = (embeddings / norms).astype(np.float32)
        
        if FAISS_AVAILABLE:
            self.index.add(normalized)
        else:
            self.vectors.extend(normalized)
    
    def search(self, query_embedding: np.ndarray, k: int = 5) -> List[Tuple[Document, float]]:
        """搜尋最相似的文件"""
        # 正規化查詢向量
        query_norm = query_embedding / np.linalg.norm(query_embedding)
        query_norm = query_norm.astype(np.float32).reshape(1, -1)
        
        if FAISS_AVAILABLE:
            scores, indices = self.index.search(query_norm, k)
            results = []
            for score, idx in zip(scores[0], indices[0]):
                if idx < len(self.documents):
                    results.append((self.documents[idx], float(score)))
        else:
            # 簡化實作：計算所有相似度
            vectors = np.array(self.vectors)
            scores = np.dot(vectors, query_norm.T).flatten()
            top_k_indices = np.argsort(scores)[::-1][:k]
            results = [(self.documents[i], float(scores[i])) for i in top_k_indices]
        
        return results
    
    def __len__(self):
        return len(self.documents)

print(f"向量儲存類別已定義")
print(f"使用 FAISS: {FAISS_AVAILABLE}")

## Part 5: 完整 RAG Pipeline

### 5.1 建立知識庫

In [ ]:
# 準備示範知識庫
knowledge_base = """
# PyTorch 深度學習框架

PyTorch 是由 Facebook AI Research 開發的開源深度學習框架。它提供了兩個高級功能：張量計算（類似 NumPy）並具有強大的 GPU 加速能力，以及建立在基於磁帶的自動微分系統上的深度神經網路。

PyTorch 的主要特點包括：
1. 動態計算圖：PyTorch 使用動態計算圖，這意味著圖形是在運行時建立的，使得除錯更加直觀。
2. Pythonic：PyTorch 的設計理念是盡可能像 Python，使得學習曲線較平緩。
3. 強大的生態系統：包括 torchvision、torchaudio、torchtext 等擴展庫。

# Transformer 架構

Transformer 是 2017 年由 Google 在論文「Attention Is All You Need」中提出的架構。它完全基於自注意力機制，摒棄了傳統的循環結構。

Transformer 的核心組件：
1. Self-Attention：允許模型在處理序列時關注不同位置的資訊。
2. Multi-Head Attention：並行執行多個注意力函數，捕捉不同的表示子空間。
3. Position Encoding：由於沒有循環結構，需要額外加入位置資訊。
4. Feed-Forward Network：每個位置獨立處理的全連接網路。

# BERT 模型

BERT（Bidirectional Encoder Representations from Transformers）是 Google 在 2018 年發表的預訓練語言模型。

BERT 的創新之處：
1. 雙向編碼：不同於 GPT 的單向，BERT 可以同時看到左右兩邊的上下文。
2. Masked Language Model：訓練時隨機遮蔽一些 token，讓模型預測。
3. Next Sentence Prediction：學習判斷兩個句子是否連續。

BERT 可用於多種下游任務：文本分類、命名實體識別、問答系統等。

# GPT 系列模型

GPT（Generative Pre-trained Transformer）是 OpenAI 開發的自回歸語言模型系列。

GPT 的特點：
1. 自回歸生成：每次生成一個 token，基於之前所有的 token。
2. 因果注意力：只能看到左側的 context，適合生成任務。
3. 規模擴展：從 GPT-1 的 1.17 億參數到 GPT-3 的 1750 億參數。

GPT-3 展示了大型語言模型的 few-shot 學習能力，無需微調即可完成多種任務。

# 知識蒸餾

知識蒸餾（Knowledge Distillation）是一種模型壓縮技術，將大型模型（教師）的知識轉移到小型模型（學生）。

蒸餾的過程：
1. 使用教師模型生成軟標籤（soft labels）。
2. 學生模型同時學習硬標籤和軟標籤。
3. 使用溫度參數控制軟化程度。

蒸餾損失函數：L = α * L_hard + (1-α) * L_soft

# RAG 系統

RAG（Retrieval-Augmented Generation）結合了檢索和生成，讓語言模型能夠存取外部知識。

RAG 的工作流程：
1. 將文件切分成小塊並建立向量索引。
2. 收到查詢時，檢索最相關的文件塊。
3. 將檢索到的內容與問題一起輸入 LLM。
4. LLM 基於提供的上下文生成回答。

RAG 的優勢：減少幻覺、支援即時更新知識、可解釋性更強。
"""

print(f"知識庫字數: {len(knowledge_base)}")

In [ ]:
# 建立 RAG 系統
class SimpleRAG:
    """簡單的 RAG 系統"""
    
    def __init__(self, embedding_model: EmbeddingModel, 
                 chunk_size: int = 300, chunk_overlap: int = 50):
        self.embedding_model = embedding_model
        self.splitter = RecursiveTextSplitter(chunk_size, chunk_overlap)
        self.vector_store = VectorStore(embedding_model.dimension)
        
    def index_documents(self, text: str, source: str = "unknown"):
        """索引文件"""
        # 切分文件
        chunks = self.splitter.split_text(text)
        
        # 建立 Document 物件
        documents = [
            Document(content=chunk, metadata={"source": source, "chunk_id": i})
            for i, chunk in enumerate(chunks)
        ]
        
        # 計算嵌入
        embeddings = self.embedding_model.embed_documents([doc.content for doc in documents])
        
        # 加入向量儲存
        self.vector_store.add_documents(documents, embeddings)
        
        print(f"已索引 {len(documents)} 個文件塊")
        return documents
    
    def retrieve(self, query: str, k: int = 3) -> List[Tuple[Document, float]]:
        """檢索相關文件"""
        query_embedding = self.embedding_model.embed_query(query)
        results = self.vector_store.search(query_embedding, k)
        return results
    
    def format_context(self, retrieved_docs: List[Tuple[Document, float]]) -> str:
        """格式化檢索到的文件為上下文"""
        context_parts = []
        for i, (doc, score) in enumerate(retrieved_docs, 1):
            context_parts.append(f"[文件 {i}] (相關度: {score:.3f})\n{doc.content}")
        return "\n\n".join(context_parts)
    
    def create_prompt(self, query: str, context: str) -> str:
        """建立 RAG prompt"""
        return f"""基於以下參考資料回答問題。如果資料中沒有相關資訊，請說明。

參考資料：
{context}

問題：{query}

回答："""

# 建立 RAG 系統
rag = SimpleRAG(embedding_model, chunk_size=400, chunk_overlap=50)

# 索引知識庫
indexed_docs = rag.index_documents(knowledge_base, source="deep_learning_notes")

In [ ]:
# 測試檢索
test_queries = [
    "什麼是 Transformer？",
    "BERT 和 GPT 有什麼區別？",
    "如何進行知識蒸餾？",
    "RAG 系統的工作流程是什麼？"
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"查詢: {query}")
    print('='*60)
    
    results = rag.retrieve(query, k=2)
    
    for i, (doc, score) in enumerate(results, 1):
        print(f"\n[結果 {i}] 相關度: {score:.3f}")
        print(f"內容: {doc.content[:200]}...")

In [ ]:
# 完整 RAG 流程（使用 GPT-2 作為示範生成器）
try:
    from transformers import GPT2LMHeadModel, GPT2Tokenizer
    
    # 載入生成模型
    print("載入生成模型...")
    gen_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    gen_model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
    gen_model.eval()
    
    def generate_answer(prompt: str, max_length: int = 200) -> str:
        """使用 LLM 生成回答"""
        inputs = gen_tokenizer.encode(prompt, return_tensors='pt', truncation=True, max_length=800)
        inputs = inputs.to(device)
        
        with torch.no_grad():
            outputs = gen_model.generate(
                inputs,
                max_new_tokens=max_length,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                pad_token_id=gen_tokenizer.eos_token_id
            )
        
        response = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)
        # 只返回生成的部分
        return response[len(prompt):].strip()
    
    # 完整 RAG 問答
    def rag_query(query: str, k: int = 3) -> str:
        """執行完整 RAG 流程"""
        # 1. 檢索
        retrieved = rag.retrieve(query, k)
        
        # 2. 格式化上下文
        context = rag.format_context(retrieved)
        
        # 3. 建立 prompt
        prompt = rag.create_prompt(query, context)
        
        # 4. 生成回答
        answer = generate_answer(prompt, max_length=150)
        
        return answer, retrieved
    
    # 測試完整流程
    query = "什麼是 BERT 模型？它有什麼特點？"
    print(f"問題: {query}\n")
    
    answer, sources = rag_query(query)
    
    print("檢索到的來源:")
    for i, (doc, score) in enumerate(sources, 1):
        print(f"  [{i}] (相關度 {score:.3f}): {doc.content[:100]}...")
    
    print(f"\n生成的回答:\n{answer}")
    
except ImportError:
    print("需要安裝 transformers 才能執行生成")

## Part 6: 評估 RAG 系統

### 6.1 檢索品質評估

In [ ]:
def evaluate_retrieval(rag_system, test_cases: List[Dict]) -> Dict:
    """
    評估檢索品質
    
    Args:
        rag_system: RAG 系統
        test_cases: [{"query": ..., "relevant_keywords": [...]}]
    
    Returns:
        評估指標
    """
    results = {
        "precision_at_k": [],
        "recall_at_k": [],
        "mrr": []  # Mean Reciprocal Rank
    }
    
    for case in test_cases:
        query = case["query"]
        relevant_keywords = case["relevant_keywords"]
        
        # 檢索
        retrieved = rag_system.retrieve(query, k=5)
        
        # 計算 Precision@K
        relevant_count = 0
        first_relevant_rank = None
        
        for rank, (doc, score) in enumerate(retrieved, 1):
            content_lower = doc.content.lower()
            is_relevant = any(kw.lower() in content_lower for kw in relevant_keywords)
            
            if is_relevant:
                relevant_count += 1
                if first_relevant_rank is None:
                    first_relevant_rank = rank
        
        k = len(retrieved)
        precision = relevant_count / k if k > 0 else 0
        recall = relevant_count / len(relevant_keywords) if relevant_keywords else 0
        mrr = 1 / first_relevant_rank if first_relevant_rank else 0
        
        results["precision_at_k"].append(precision)
        results["recall_at_k"].append(recall)
        results["mrr"].append(mrr)
    
    # 計算平均
    return {
        "avg_precision": np.mean(results["precision_at_k"]),
        "avg_recall": np.mean(results["recall_at_k"]),
        "mrr": np.mean(results["mrr"]),
        "details": results
    }

# 準備測試案例
test_cases = [
    {
        "query": "什麼是 Transformer 的核心組件？",
        "relevant_keywords": ["transformer", "attention", "self-attention"]
    },
    {
        "query": "BERT 如何進行預訓練？",
        "relevant_keywords": ["bert", "masked", "預訓練", "mlm"]
    },
    {
        "query": "知識蒸餾的損失函數是什麼？",
        "relevant_keywords": ["蒸餾", "distillation", "損失", "教師", "學生"]
    },
    {
        "query": "RAG 有什麼優勢？",
        "relevant_keywords": ["rag", "檢索", "幻覺", "retrieval"]
    }
]

# 評估
eval_results = evaluate_retrieval(rag, test_cases)

print("RAG 檢索評估結果:")
print(f"  平均 Precision@K: {eval_results['avg_precision']:.3f}")
print(f"  平均 Recall@K: {eval_results['avg_recall']:.3f}")
print(f"  MRR (Mean Reciprocal Rank): {eval_results['mrr']:.3f}")

## Part 7: 練習題

### Exercise 1: 實作語義切分器

使用 embedding 相似度來決定切分點。

In [ ]:
class SemanticTextSplitter:
    """
    基於語義相似度的文本切分器
    
    想法：當連續句子的語義相似度低於閾值時，在該處切分
    """
    
    def __init__(self, embedding_model: EmbeddingModel, 
                 similarity_threshold: float = 0.5,
                 min_chunk_size: int = 100):
        self.embedding_model = embedding_model
        self.similarity_threshold = similarity_threshold
        self.min_chunk_size = min_chunk_size
    
    def split_text(self, text: str) -> List[str]:
        """基於語義的切分"""
        # TODO: 實作語義切分
        # 步驟：
        # 1. 先按句子切分
        # 2. 計算每個句子的嵌入
        # 3. 計算相鄰句子的相似度
        # 4. 在相似度低於閾值的地方切分
        # 5. 合併太短的 chunk
        
        # 步驟 1: 按句子切分
        sentences = re.split(r'(?<=[.!?。！？])\s+', text)
        sentences = [s.strip() for s in sentences if s.strip()]
        
        if len(sentences) <= 1:
            return [text]
        
        # 步驟 2: 計算嵌入
        embeddings = self.embedding_model.embed_documents(sentences)
        
        # 步驟 3: 計算相鄰相似度
        similarities = []
        for i in range(len(embeddings) - 1):
            sim = np.dot(embeddings[i], embeddings[i+1]) / (
                np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[i+1])
            )
            similarities.append(sim)
        
        # 步驟 4: 找切分點
        chunks = []
        current_chunk = [sentences[0]]
        current_length = len(sentences[0])
        
        for i, (sentence, sim) in enumerate(zip(sentences[1:], similarities)):
            if sim < self.similarity_threshold and current_length >= self.min_chunk_size:
                # 切分
                chunks.append(' '.join(current_chunk))
                current_chunk = [sentence]
                current_length = len(sentence)
            else:
                current_chunk.append(sentence)
                current_length += len(sentence)
        
        if current_chunk:
            chunks.append(' '.join(current_chunk))
        
        return chunks

# 測試語義切分
semantic_splitter = SemanticTextSplitter(embedding_model, similarity_threshold=0.4)
semantic_chunks = semantic_splitter.split_text(knowledge_base)

print(f"語義切分產生 {len(semantic_chunks)} 個 chunks")
for i, chunk in enumerate(semantic_chunks[:3]):
    print(f"\nChunk {i+1} ({len(chunk)} chars):")
    print(f"  {chunk[:150]}...")

### Exercise 2: 實作 Hybrid Search（混合搜尋）

結合關鍵字搜尋（BM25）和向量搜尋。

In [ ]:
class BM25:
    """簡化的 BM25 實作"""
    
    def __init__(self, k1: float = 1.5, b: float = 0.75):
        self.k1 = k1
        self.b = b
        self.doc_lengths = []
        self.avg_doc_length = 0
        self.doc_freqs = {}  # term -> doc count
        self.term_freqs = []  # list of {term: freq} for each doc
        self.N = 0
    
    def _tokenize(self, text: str) -> List[str]:
        """簡單的分詞"""
        return re.findall(r'\w+', text.lower())
    
    def fit(self, documents: List[str]):
        """建立索引"""
        self.N = len(documents)
        self.term_freqs = []
        self.doc_lengths = []
        
        for doc in documents:
            tokens = self._tokenize(doc)
            self.doc_lengths.append(len(tokens))
            
            # 計算詞頻
            tf = {}
            for token in tokens:
                tf[token] = tf.get(token, 0) + 1
            self.term_freqs.append(tf)
            
            # 更新文件頻率
            for token in set(tokens):
                self.doc_freqs[token] = self.doc_freqs.get(token, 0) + 1
        
        self.avg_doc_length = sum(self.doc_lengths) / self.N
    
    def score(self, query: str, doc_idx: int) -> float:
        """計算 BM25 分數"""
        query_tokens = self._tokenize(query)
        score = 0
        
        doc_length = self.doc_lengths[doc_idx]
        tf_doc = self.term_freqs[doc_idx]
        
        for term in query_tokens:
            if term not in tf_doc:
                continue
            
            tf = tf_doc[term]
            df = self.doc_freqs.get(term, 0)
            
            # IDF
            idf = np.log((self.N - df + 0.5) / (df + 0.5) + 1)
            
            # TF 正規化
            tf_norm = (tf * (self.k1 + 1)) / (
                tf + self.k1 * (1 - self.b + self.b * doc_length / self.avg_doc_length)
            )
            
            score += idf * tf_norm
        
        return score
    
    def search(self, query: str, k: int = 5) -> List[Tuple[int, float]]:
        """搜尋最相關的文件"""
        scores = [(i, self.score(query, i)) for i in range(self.N)]
        scores.sort(key=lambda x: x[1], reverse=True)
        return scores[:k]

class HybridRAG:
    """混合搜尋 RAG"""
    
    def __init__(self, embedding_model: EmbeddingModel, alpha: float = 0.5):
        """
        Args:
            alpha: 向量搜尋的權重（0-1），1-alpha 為 BM25 權重
        """
        self.embedding_model = embedding_model
        self.alpha = alpha
        self.vector_store = VectorStore(embedding_model.dimension)
        self.bm25 = BM25()
        self.documents: List[Document] = []
    
    def add_documents(self, documents: List[Document]):
        """加入文件"""
        self.documents.extend(documents)
        
        # 向量索引
        embeddings = self.embedding_model.embed_documents([d.content for d in documents])
        self.vector_store.add_documents(documents, embeddings)
        
        # BM25 索引
        self.bm25.fit([d.content for d in self.documents])
    
    def search(self, query: str, k: int = 5) -> List[Tuple[Document, float]]:
        """混合搜尋"""
        # 向量搜尋
        query_embedding = self.embedding_model.embed_query(query)
        vector_results = self.vector_store.search(query_embedding, k=k*2)
        
        # BM25 搜尋
        bm25_results = self.bm25.search(query, k=k*2)
        
        # 正規化分數
        vector_scores = {id(r[0]): r[1] for r in vector_results}
        max_vector = max(vector_scores.values()) if vector_scores else 1
        vector_scores = {k: v/max_vector for k, v in vector_scores.items()}
        
        bm25_scores = {bm25_results[i][0]: bm25_results[i][1] for i in range(len(bm25_results))}
        max_bm25 = max(bm25_scores.values()) if bm25_scores else 1
        bm25_scores = {k: v/max_bm25 for k, v in bm25_scores.items()}
        
        # 合併分數
        combined = {}
        for doc, _ in vector_results:
            doc_id = id(doc)
            idx = self.documents.index(doc) if doc in self.documents else -1
            v_score = vector_scores.get(doc_id, 0)
            b_score = bm25_scores.get(idx, 0)
            combined[doc_id] = (doc, self.alpha * v_score + (1-self.alpha) * b_score)
        
        # 排序並返回
        results = sorted(combined.values(), key=lambda x: x[1], reverse=True)[:k]
        return results

# 測試混合搜尋
hybrid_rag = HybridRAG(embedding_model, alpha=0.7)

# 使用之前切分的文件
docs = [Document(content=chunk) for chunk in chunks_sentences[:10]]
hybrid_rag.add_documents(docs)

# 搜尋
query = "machine learning neural networks"
results = hybrid_rag.search(query, k=3)

print(f"混合搜尋結果 (query: '{query}'):")
for i, (doc, score) in enumerate(results, 1):
    print(f"\n[{i}] Score: {score:.3f}")
    print(f"    {doc.content[:100]}...")

## 總結

```
┌─────────────────────────────────────────────────────────────┐
│                    RAG 基礎總結                              │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  1. RAG 解決的問題                                          │
│     • 知識截斷、幻覺、私有資料存取                           │
│                                                             │
│  2. 核心組件                                                │
│     • 文件切分（固定/句子/遞迴/語義）                        │
│     • 向量嵌入（Sentence Transformers）                     │
│     • 向量資料庫（FAISS）                                   │
│     • 檢索與生成整合                                        │
│                                                             │
│  3. 評估指標                                                │
│     • Precision@K、Recall@K、MRR                           │
│                                                             │
│  4. 進階技術                                                │
│     • 語義切分                                              │
│     • 混合搜尋（向量 + BM25）                               │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### 下一步學習

- **RAG 進階**: `ai_agents/rag_fundamentals.ipynb` (Reranking, HyDE)
- **AI Agent**: `ai_agents/agent_tools.ipynb`
- **LLM 微調**: `language_models/llm_finetuning.ipynb`

## 參考資源

### 課程
- [李宏毅 2025 Fall GenAI-ML HW2](https://speech.ee.ntu.edu.tw/~hylee/GenAI-ML/2025-fall.php)
- [李宏毅 2025 Spring ML HW1](https://speech.ee.ntu.edu.tw/~hylee/ml/2025-spring.php)

### 論文
- [Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks](https://arxiv.org/abs/2005.11401)

### 工具
- [FAISS](https://github.com/facebookresearch/faiss)
- [Sentence Transformers](https://www.sbert.net/)
- [LangChain](https://python.langchain.com/)